In [127]:

"""
now, let's compute the equilibrium with global supply chains
we first suppose that type-I supply chains use south unskilled labor and north unskilled labor simueltaneously. 
so we must have:
chi*w=w_u>w
this simplifies our calculation a lot.
"""
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import fsolve
from scipy import integrate
import scipy.optimize

rho=0.03 #subjective discount rate
eta=0.4 #north innovation param
theta=0.2 #south imitation param
gamma=1.5 #quality ladder
L_u=0.4 #north unskilled labor proportion
lambda_u=0.5 #unskilled mistake rate
lambda_s=0.25 #skilled mistake rate
chi=1.3 #offshoring cost with tech gaps in type-I GSCs
beta=0.5 #north bargaining power upon surplus in type-II GSCs

#the core algorithm solving the assignment problem of two types of labor
def s_star(w_u, w_s):
    temp1=(w_s/lambda_s-w_u/lambda_u)*lambda_s
    temp2=(lambda_u-lambda_s)*w_u/lambda_u
    if temp1<=temp2:
        return 0
    elif temp1>=temp2*np.exp(lambda_u):
        return 1
    else:
        return np.log(temp1/temp2)/lambda_u
def lu1(sstar):
    return (np.exp((lambda_u-lambda_s)*sstar)-np.exp(-lambda_s*sstar))*np.exp(lambda_s)/lambda_u
def ls1(sstar):
    return (np.exp(-lambda_s*sstar)-np.exp(-lambda_s))*np.exp(lambda_s)/lambda_s
def realcost(w_u, w_s):
    return w_u*lu1(s_star(w_u, w_s))+w_s*ls1(s_star(w_u, w_s))
def reallu1(w_u, w_s):
    return lu1(s_star(w_u, w_s))
def realls1(w_u, w_s):
    return ls1(s_star(w_u, w_s))

#type-II GSC cost and labor demand
def typeIIcost(w, w_s):
    return realcost(w, w_s)
def typeIIl1(w, w_s):
    return reallu1(w, w_s)
def typeIIls1(w, w_s):
    return realls1(w, w_s)

#type-I GSC cost and labor demand
#note, since chi*w=w_u, so the demand for south labor and north unskilled is the same
def typeIcost(w_u, w_s):
    return realcost(w_u, w_s)
def typeIll1(w_u, w_s, typeIlu1):
    return reallu1(w_u, w_s)-typeIlu1
def typeIls1(w_u, w_s):
    return realls1(w_u, w_s)

#now, we can express profits as functions of wages
def pi_s(w, w_u, w_s):
    return 1-typeIIcost(w, 100)/typeIIcost(w_u, w_s)
def piII(w, w_u, w_s):
    return 1-typeIIcost(w, w_s)/typeIIcost(w_u, w_s)
def pi_nII(w, w_u, w_s):
    return beta*(piII(w, w_u, w_s)-pi_s(w, w_u, w_s))
def pi_sII(w, w_u, w_s):
    return pi_s(w, w_u, w_s)+(1-beta)*(piII(w, w_u, w_s)-pi_s(w, w_u, w_s))
def pi_nI(w, w_u, w_s):
    return 1-typeIcost(w_u, w_s)/(gamma*typeIIcost(w, 100))

#then, research intensity and industrial share
def n_inn(w, w_u, w_s):
    return (pi_sII(w, w_u, w_s)*theta/w-rho)/eta
def n_imi(w, w_u, w_s):
    temp1=pi_nI(w, w_u, w_s)-rho*w_s/eta
    temp2=theta*(w_s/eta-pi_nII(w, w_u, w_s)/(rho+eta*n_inn(w, w_u, w_s)))
    return temp1/temp2
def xi_I(w, w_u, w_s):
    return eta*n_inn(w, w_u, w_s)/(eta*n_inn(w, w_u, w_s)+theta*n_imi(w, w_u, w_s))

#excess demand for labors
def southlaborexcessdemand(w, w_u, w_s, typeIlu1):
    temp1=xi_I(w, w_u, w_s)*n_imi(w, w_u, w_s)+xi_I(w, w_u, w_s)*typeIll1(w_u, w_s, typeIlu1)/(gamma*typeIIcost(w, 100))
    temp2=(1-xi_I(w, w_u, w_s))*typeIIl1(w, w_s)/typeIIcost(w_u, w_s)
    return temp1+temp2-1
def northunskillexcessdemand(w, w_u, w_s, typeIlu1):
    return xi_I(w, w_u, w_s)*typeIlu1/(gamma*typeIIcost(w, 100))-L_u
def northskillexcessdemand(w, w_u, w_s, typeIlu1):
    temp1=xi_I(w, w_u, w_s)*typeIls1(w_u, w_s)/(gamma*typeIIcost(w, 100))
    temp2=(1-xi_I(w, w_u, w_s))*n_inn(w, w_u, w_s)+(1-xi_I(w, w_u, w_s))*typeIIls1(w, w_s)/typeIIcost(w_u, w_s)
    return temp1+temp2-(1-L_u)

#now, we minimize the quadratic sum of those excess demands
def tol(x):
    return southlaborexcessdemand(x[0], x[1], x[2], x[3])**2+northunskillexcessdemand(x[0], x[1], x[2], x[3])**2+northskillexcessdemand(x[0], x[1], x[2], x[3])**2
gscwage=scipy.optimize.minimize(tol, (0.4, 0.5, 1.1, 0.2), constraints=({'type': 'eq', 'fun': lambda x: x[1]-chi*x[0]},
                                                         {'type': 'ineq', 'fun': lambda x: x[2]-x[1]},
                                                         {'type': 'ineq', 'fun': lambda x: x[0]},
                                                         {'type': 'ineq', 'fun': lambda x: x[3]},
                                                         {'type': 'eq', 'fun': lambda x: northunskillexcessdemand(x[0], x[1], x[2], x[3])},
                                                         {'type': 'eq', 'fun': lambda x: southlaborexcessdemand(x[0], x[1], x[2], x[3])},
                                                         {'type': 'eq', 'fun': lambda x: northskillexcessdemand(x[0], x[1], x[2], x[3])},                                                                      
                                                         {'type': 'ineq', 'fun': lambda x: 1-x[0]}))
print("equilibrium under global supply chains", "\n",
      "min result ", gscwage.fun, "\n",
     "south (unskilled) wage ", gscwage.x[0], "\n",
     "north unskilled wage ", gscwage.x[1], "\n",
     "north skilled wage ", gscwage.x[2], "\n",
     "southlaborexcessdemand ", southlaborexcessdemand(gscwage.x[0], gscwage.x[1], gscwage.x[2], gscwage.x[3]), "\n",
     "northunskillexcessdemand ", northunskillexcessdemand(gscwage.x[0], gscwage.x[1], gscwage.x[2], gscwage.x[3]), "\n",
     "northskillexcessdemand ", northskillexcessdemand(gscwage.x[0], gscwage.x[1], gscwage.x[2], gscwage.x[3]), "\n",
     "north innovate intensity ", n_inn(gscwage.x[0], gscwage.x[1], gscwage.x[2]), "\n",
     "south imitation intensity ", n_imi(gscwage.x[0], gscwage.x[1], gscwage.x[2]), "\n",
     "north industrial share ", xi_I(gscwage.x[0], gscwage.x[1], gscwage.x[2]), "\n",
     "south industrial share ", 1-xi_I(gscwage.x[0], gscwage.x[1], gscwage.x[2]))

equilibrium under global supply chains 
 min result  0.134592000511 
 south (unskilled) wage  0.435740600455 
 north unskilled wage  0.566462780491 
 north skilled wage  0.503028148031 
 southlaborexcessdemand  0.020634075166 
 northunskillexcessdemand  -0.299280826707 
 northskillexcessdemand  0.21118054413 
 north innovate intensity  -0.0455233170004 
 south imitation intensity  -151.540401264 
 north industrial share  0.000600446907703 
 south industrial share  0.999399553092


In [112]:
0.6244/0.5637

1.1076813908107148

In [ ]:
print("equilibrium under global supply chains", "\n",
      "min result ", gscwage.fun, "\n",
     "south (unskilled) wage ", gscwage.x[0], "\n",
     "north unskilled wage ", gscwage.x[1], "\n",
     "north skilled wage ", gscwage.x[2], "\n",
     "north innovate intensity ", n_inn(gscwage.x[0], gscwage.x[1], gscwage.x[2]), "\n",
     "south imitation intensity ", n_imi(gscwage.x[0], gscwage.x[1], gscwage.x[2]), "\n",
     "north industrial share ", xi_I(gscwage.x[0], gscwage.x[1], gscwage.x[2]), "\n",
     "south industrial share ", 1-xi_I(gscwage.x[0], gscwage.x[1], gscwage.x[2]))

In [121]:
"""
now, consider where chi*w>w_u>=w
"""
rho=0.03 #subjective discount rate
eta=0.4 #north innovation param
theta=0.2 #south imitation param
gamma=1.5 #quality ladder
L_u=0.4 #north unskilled labor proportion
lambda_u=0.5 #unskilled mistake rate
lambda_s=0.25 #skilled mistake rate
chi=1.3 #offshoring cost with tech gaps in type-I GSCs
beta=0.5 #north bargaining power upon surplus in type-II GSCs

#the core algorithm solving the assignment problem of two types of labor
def s_star(w_u, w_s):
    temp1=(w_s/lambda_s-w_u/lambda_u)*lambda_s
    temp2=(lambda_u-lambda_s)*w_u/lambda_u
    if temp1<=temp2:
        return 0
    elif temp1>=temp2*np.exp(lambda_u):
        return 1
    else:
        return np.log(temp1/temp2)/lambda_u
def lu1(sstar):
    return (np.exp((lambda_u-lambda_s)*sstar)-np.exp(-lambda_s*sstar))*np.exp(lambda_s)/lambda_u
def ls1(sstar):
    return (np.exp(-lambda_s*sstar)-np.exp(-lambda_s))*np.exp(lambda_s)/lambda_s
def realcost(w_u, w_s):
    return w_u*lu1(s_star(w_u, w_s))+w_s*ls1(s_star(w_u, w_s))
def reallu1(w_u, w_s):
    return lu1(s_star(w_u, w_s))
def realls1(w_u, w_s):
    return ls1(s_star(w_u, w_s))

#type-II GSC cost and labor demand
def typeIIcost(w, w_s):
    return realcost(w, w_s)
def typeIIl1(w, w_s):
    return reallu1(w, w_s)
def typeIIls1(w, w_s):
    return realls1(w, w_s)

#type-I GSC cost and labor demand
#note, since chi*w=w_u, so the demand for south labor and north unskilled is the same
def typeIcost(w_u, w_s):
    return realcost(w_u, w_s)
def typeIlu1(w_u, w_s):
    return reallu1(w_u, w_s)
def typeIls1(w_u, w_s):
    return realls1(w_u, w_s)

#now, we can express profits as functions of wages
def pi_s(w, w_u, w_s):
    return 1-typeIIcost(w, 100)/typeIIcost(w_u, w_s)
def piII(w, w_u, w_s):
    return 1-typeIIcost(w, w_s)/typeIIcost(w_u, w_s)
def pi_nII(w, w_u, w_s):
    return beta*(piII(w, w_u, w_s)-pi_s(w, w_u, w_s))
def pi_sII(w, w_u, w_s):
    return pi_s(w, w_u, w_s)+(1-beta)*(piII(w, w_u, w_s)-pi_s(w, w_u, w_s))
def pi_nI(w, w_u, w_s):
    return 1-typeIcost(w_u, w_s)/(gamma*typeIIcost(w, 100))

#then, research intensity and industrial share
def n_inn(w, w_u, w_s):
    return (pi_sII(w, w_u, w_s)*theta/w-rho)/eta
def n_imi(w, w_u, w_s):
    temp1=pi_nI(w, w_u, w_s)-rho*w_s/eta
    temp2=theta*(w_s/eta-pi_nII(w, w_u, w_s)/(rho+eta*n_inn(w, w_u, w_s)))
    return temp1/temp2
def xi_I(w, w_u, w_s):
    return eta*n_inn(w, w_u, w_s)/(eta*n_inn(w, w_u, w_s)+theta*n_imi(w, w_u, w_s))

#excess demand for labors
def southlaborexcessdemand(w, w_u, w_s):
    temp1=xi_I(w, w_u, w_s)*n_imi(w, w_u, w_s)
    temp2=(1-xi_I(w, w_u, w_s))*typeIIl1(w, w_s)/typeIIcost(w_u, w_s)
    return temp1+temp2-1
def northunskillexcessdemand(w, w_u, w_s):
    return xi_I(w, w_u, w_s)*typeIlu1(w_u, w_s)/(gamma*typeIIcost(w, 100))-L_u
def northskillexcessdemand(w, w_u, w_s):
    temp1=xi_I(w, w_u, w_s)*typeIls1(w_u, w_s)/(gamma*typeIIcost(w, 100))
    temp2=(1-xi_I(w, w_u, w_s))*n_inn(w, w_u, w_s)+(1-xi_I(w, w_u, w_s))*typeIIls1(w, w_s)/typeIIcost(w_u, w_s)
    return temp1+temp2-(1-L_u)

#now, we minimize the quadratic sum of those excess demands
def tol(x):
    return southlaborexcessdemand(x[0], x[1], x[2])**2+northunskillexcessdemand(x[0], x[1], x[2])**2+northskillexcessdemand(x[0], x[1], x[2])**2
gscwage=scipy.optimize.minimize(tol, (0.5, 0.6, 1.1), constraints=({'type': 'ineq', 'fun': lambda x: chi*x[0]-x[1]-0.05},
                                                         {'type': 'ineq', 'fun': lambda x: x[2]-x[1]},
                                                         {'type': 'ineq', 'fun': lambda x: x[0]},
                                                         {'type': 'ineq', 'fun': lambda x: x[1]},
                                                         {'type': 'ineq', 'fun': lambda x: x[2]},
                                                         {'type': 'ineq', 'fun': lambda x: 1-x[0]},
                                                         {'type': 'ineq', 'fun': lambda x: n_inn(x[0], x[1], x[2])-0.01},
                                                         {'type': 'ineq', 'fun': lambda x: n_imi(x[0], x[1], x[2])-0.01}))
print("equilibrium under global supply chains", "\n",
      "min result ", gscwage.fun, "\n",
     "south (unskilled) wage ", gscwage.x[0], "\n",
     "north unskilled wage ", gscwage.x[1], "\n",
     "north skilled wage ", gscwage.x[2], "\n",
     "southlaborexcessdemand ", southlaborexcessdemand(gscwage.x[0], gscwage.x[1], gscwage.x[2]), "\n",
     "northunskillexcessdemand ", northunskillexcessdemand(gscwage.x[0], gscwage.x[1], gscwage.x[2]), "\n",
     "northskillexcessdemand ", northskillexcessdemand(gscwage.x[0], gscwage.x[1], gscwage.x[2]), "\n",
     "north innovate intensity ", n_inn(gscwage.x[0], gscwage.x[1], gscwage.x[2]), "\n",
     "south imitation intensity ", n_imi(gscwage.x[0], gscwage.x[1], gscwage.x[2]), "\n",
     "north industrial share ", xi_I(gscwage.x[0], gscwage.x[1], gscwage.x[2]), "\n",
     "south industrial share ", 1-xi_I(gscwage.x[0], gscwage.x[1], gscwage.x[2]))

equilibrium under global supply chains 
 min result  0.253731269128 
 south (unskilled) wage  0.587946123246 
 north unskilled wage  0.714329960222 
 north skilled wage  0.882244214617 
 southlaborexcessdemand  0.0525261800978 
 northunskillexcessdemand  -0.120384629704 
 northskillexcessdemand  -0.486291898414 
 north innovate intensity  0.0698959596416 
 south imitation intensity  0.295267351806 
 north industrial share  0.321316952822 
 south industrial share  0.678683047178
